In [ ]:
# 库安装命令 (请在终端中运行以下命令):
# pip install google-cloud-firestore \
#     langchain-google-firestore \
#     langchain-openai \
#     openai

In [ ]:
# 导入 Firestore 客户端库
from google.cloud import firestore
# 导入 Langchain Firestore Chat 消息历史记录类
from langchain_google_firestore import FirestoreChatMessageHistory
# 导入 Langchain OpenAI Chat 模型
from langchain_openai import ChatOpenAI

In [ ]:
"""
步骤来复现这个例子:
1. 创建一个 Firebase 账户
2. 创建一个新的 Firebase 项目
    - 复制项目 ID
3. 在 Firebase 项目中创建一个 Firestore 数据库
4. 在你的电脑上安装 Google Cloud CLI
    - https://cloud.google.com/sdk/docs/install
    - 使用你的 Google 账户验证 Google Cloud CLI
        - https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev
    - 将你的默认项目设置为你创建的新 Firebase 项目
5. 在 Google Cloud 控制台中启用 Firestore API:
    - https://console.cloud.google.com/apis/enableflow?apiid=firestore.googleapis.com&project=crewai-automation
"""

In [ ]:
# 设置 Firebase Firestore 项目 ID
PROJECT_ID = "langchain-demo-abf48"
# 设置会话 ID, 用于区分不同的用户或会话，这里使用 "user_session_new" 作为示例
SESSION_ID = "user_session_new"  # This could be a username or a unique ID
# 设置 Firestore 集合名称，用于存储聊天历史记录，这里使用 "chat_history" 作为示例
COLLECTION_NAME = "chat_history"

# 初始化 Firestore 客户端
print("Initializing Firestore Client...")
# 创建 Firestore 客户端对象，需要传入项目 ID
client = firestore.Client(project=PROJECT_ID)

# 初始化 Firestore Chat 消息历史记录
print("Initializing Firestore Chat Message History...")
# 创建 FirestoreChatMessageHistory 对象，用于管理聊天历史记录的存储和检索
chat_history = FirestoreChatMessageHistory(
    session_id=SESSION_ID, # 传入会话 ID，用于区分不同会话的数据
    collection=COLLECTION_NAME, # 传入集合名称，指定数据存储在哪个集合中
    client=client, # 传入 Firestore 客户端对象，用于与 Firestore 数据库交互
)
print("Chat History Initialized.")
# 打印当前的聊天历史记录，刚初始化时应该是空的
print("Current Chat History:", chat_history.messages)

# 初始化 Chat 模型
model = ChatOpenAI()
print("Start chatting with the AI. Type 'exit' to quit.")
# 进入聊天循环
while True:
    # 获取用户输入
    human_input = input("User: ")
    # 如果用户输入 "exit"，则退出循环
    if human_input.lower() == "exit":
        break

    # 将用户消息添加到聊天历史记录中
    chat_history.add_user_message(human_input)

    # 使用 Chat 模型处理聊天历史记录，并获取 AI 的回复
    ai_response = model.invoke(chat_history.messages)
    # 将 AI 的回复添加到聊天历史记录中
    chat_history.add_ai_message(ai_response.content)

    # 打印 AI 的回复
    print(f"AI: {ai_response.content}")